In [1]:
import os, math
import pandas as pd

In [2]:
blogPath = "./blogs/"
pathList = os.listdir(path=blogPath)
blogSampleSize = 15

In [3]:
pathList[0].split(".")

['1000331', 'female', '37', 'indUnk', 'Leo', 'xml']

In [4]:
def getOccupation(lst):
    rtrn = []
    for person in lst:
        rtrn.append(person.split(".")[3])
    return rtrn

occupations = getOccupation(pathList)[:blogSampleSize]
occupationDict = {}
occupationCounter = 0
for occ in occupations:
    if occ not in occupationDict:
        occupationDict[occ] = str(occupationCounter)
        occupationCounter += 1

In [5]:
blogs = []
for ind in range(blogSampleSize):
    file = open(blogPath+pathList[ind], "rb")
    posts = []
    for line in file:
        text = line.decode("utf-8", errors="ignore")
        text = text.strip()
        if text != "" and text[0] != "<":
            posts.append(text)
    blogs.append(" ".join(posts))
    file.close()


In [6]:
# splits up a string into a list of words.
# str -> [str]
def getWords(str):
    words = []
    
    str = str+"."
    lastInd = 0
    for ind in range(len(str)):
        if not (str[ind].isalpha()):
            if lastInd == ind:
                lastInd += 1
                continue
            else:
                words.append(str[lastInd:ind].lower())
                lastInd = ind+1
    
    return words
            

In [7]:
dicts = []
allWordsSeen = {}
for blog in blogs:
    newDict = {}
    dicts.append(newDict)
    wordList = getWords(blog)
    for word in wordList:
#         ADD TO THIS PERSON'S DICT
        if word not in newDict:
            newDict[word] = 1
        else:
            newDict[word] += 1
#         ADD TO ALL WORDS SEEN
        if word not in allWordsSeen:
            allWordsSeen[word] = 1
        else:
            allWordsSeen[word] += 1



In [8]:
totalPeople = len(dicts)
testWordList = [word for word in allWordsSeen if ((allWordsSeen[word]/totalPeople > .50) or (allWordsSeen[word]/totalPeople < .01))]
wordsToRemove = []
for word in allWordsSeen:
    count = allWordsSeen[word]
    
    if ((count / totalPeople) > .50) or ((count / totalPeople) < .01):
#         print(word)
#         del allWordsSeen[word]
        wordsToRemove.append(word)
        for person in dicts:
            if word in person:
                del person[word]
    else:
        for person in dicts:
            if word not in person:
                person[word] = 0

for word in wordsToRemove:
    del allWordsSeen[word]

In [9]:
print([word for word in testWordList if word not in wordsToRemove])

[]


In [10]:
dictOfDicts = {}
for ind in range(blogSampleSize):
    dictOfDicts[pathList[ind]] = dicts[ind]

In [11]:
tdm = pd.DataFrame(dictOfDicts, dtype=float)
# tdm = tdm.transpose()
# tdm

In [12]:
def termFrequency(person):
    return .5 + (.5*person/person.max())

In [13]:
for word in allWordsSeen:
    word
#     personCount = 0
#     for person in tdm:
#         print(tdm[person][word])


In [14]:
tdm['1000331.female.37.indUnk.Leo.xml']["aa"]

0.0

In [15]:
# (This step scales really badly)

IDF_Dict = {}
for word in allWordsSeen:
    personCount = 0
    for person in tdm:
        if tdm[person][word] > 0:
            personCount += 1
    if personCount > 0:
        IDF_Dict[word] = math.log(blogSampleSize / personCount)

In [16]:
# IDF_Dict

In [17]:
# for person in tdm:
#     tdm[person] = termFrequency(tdm[person])
#     for i in range(len(person)):
#         tdm[person][i] = tdm[person][i] * IDF_Dict[tdm[person].keys()[i]]

In [18]:
# tdm['1000331.female.37.indUnk.Leo.xml']

In [19]:
# cosDist = sum(ser1*ser2)/(sqrt(sum(ser1^2)) * sqrt(sum(ser2^2)))
# def cosDist(ser1, ser2):
#     numerator = ser1.dot(ser2)
#     ser1Denominator = 0
#     for val in ser1:
#         ser1Denominator += math.pow(val, 2)
#     ser1Denominator = math.sqrt(ser1Denominator)
#     ser2Denominator = 0
#     for val in ser2:
#         ser2Denominator += math.pow(val, 2)
#     ser2Denominator = math.sqrt(ser2Denominator)
#     return 1 - numerator/(ser1Denominator*ser2Denominator)

def cosDist(ser1, ser2):
    numerator = ser1.dot(ser2)
    # ser1Denominator = itertools.accumulate(ser1, )
    # for val in ser1:
    #     ser1Denominator += math.pow(val, 2)
    ser1Denominator = math.sqrt(sum(map((lambda x: x**2), ser1)))
    ser2Denominator = math.sqrt(sum(map((lambda x: x**2), ser2)))
    # ser2Denominator = 0
    # for val in ser2:
    #     ser2Denominator += math.pow(val, 2)
    # ser2Denominator = math.sqrt(ser2Denominator)
    return 1 - numerator/(ser1Denominator*ser2Denominator)

In [20]:
# cosDist(tdm['1000331.female.37.indUnk.Leo.xml'], tdm['1008329.female.16.Student.Pisces.xml'])
pt1 = math.sqrt(sum(map((lambda x: x**2), tdm['1000331.female.37.indUnk.Leo.xml'])))
pt2 = math.sqrt(sum(map((lambda x: x**2), tdm['1011153.female.27.Technology.Virgo.xml'])))
pt1*pt2

136.0404351654316

In [21]:
testDict = {person:{relation:cosDist(tdm[person], tdm[relation]) for relation in tdm} for person in tdm}

In [22]:
testDict

{'1000331.female.37.indUnk.Leo.xml': {'1000331.female.37.indUnk.Leo.xml': 0.0,
  '1004904.male.23.Arts.Capricorn.xml': 1.0,
  '1005076.female.25.Arts.Cancer.xml': 1.0,
  '1005545.male.25.Engineering.Sagittarius.xml': 0.9860537004037397,
  '1007188.male.48.Religion.Libra.xml': 1.0,
  '100812.female.26.Architecture.Aries.xml': 0.979839171567941,
  '1008329.female.16.Student.Pisces.xml': 0.9935006902442041,
  '1009572.male.25.indUnk.Cancer.xml': 1.0,
  '1011153.female.27.Technology.Virgo.xml': 1.0,
  '1011289.female.25.indUnk.Libra.xml': 0.9763987258452946,
  '1011311.female.17.indUnk.Scorpio.xml': 0.9834878419305999,
  '1013637.male.17.RealEstate.Virgo.xml': 0.9850438389507422,
  '1015252.female.23.indUnk.Pisces.xml': 0.9827855153033251,
  '1015556.male.34.Technology.Virgo.xml': 0.9819159763087922,
  '1016560.male.41.Publishing.Sagittarius.xml': 0.9825961893665613},
 '1004904.male.23.Arts.Capricorn.xml': {'1000331.female.37.indUnk.Leo.xml': 1.0,
  '1004904.male.23.Arts.Capricorn.xml': 0.

In [23]:
# # (This step scales really badly)

# similarityDict = {}
# for person in tdm:
#     similarityDict[person] = {}
    
# for ind in range(len(keys)):
#     for i in range(len(keys)):
#         dist = cosDist(tdm[keys[ind]], tdm[keys[i]])
#         similarityDict[keys[ind]][keys[i]] = ((dist < .0003) and (dist > .000000001))
# #         if ((dist < .00007) and (dist > .000000001)):
# #             print(dist)
# #             print(i)
# #             print(ind)

similarityDict = {person:{relation:cosDist(tdm[person], tdm[relation]) for relation in tdm} for person in tdm}

In [24]:
similarityFrame = pd.DataFrame(similarityDict)

In [25]:
person = similarityFrame["1004904.male.23.Arts.Capricorn.xml"]
for p in person.keys():
    print(person[p])

1.0
0.0
0.985052634162647
0.9782120311465031
0.9967018605987582
0.9784088426175234
0.9613061111616283
0.9974510687849324
0.9944132334554562
0.958145633657305
0.9722861410953052
0.9382930294125026
0.9370358556764453
0.963730237806011
1.0


In [26]:
# similarityFrame.to_json(path_or_buf="./similarity.json")

In [27]:
# similarityFrame = similarityFrame.transpose()
# similarityFrame.to_csv(path_or_buf="./similarity.csv", index_label="person", na_rep="undefined")

In [28]:
file = open("writeTest.json", "w")
file.write("{")
file.write('"nodes":[')
for i in range(len(tdm.keys())-1):
    file.write('{"id": "'+tdm.keys()[i]+'", "group":'+ occupationDict[occupations[i]] +'},')
file.write('{"id": "'+tdm.keys()[len(tdm.keys())-1]+'", "group": 1'+ occupationDict[occupations[i]] +'}')
file.write("],")

file.write('"links":[')
for person in similarityFrame:
    for relation in similarityFrame[person].keys():
        if similarityFrame[person][relation]:
            file.write('{"source": "'+person+'", "target": "'+relation+'", "value": 1}, \n')
file.write("]")
file.write("}")
file.close()